## Test data used for previous project

In [23]:
import pandas as pd
import csv

In [3]:
test_data_manual = pd.read_csv("authors_to_genderize_together.csv")
test_data_algo = pd.read_csv("authors_to_genderize_together_algo.csv")

In [4]:
test_data_manual.head()

,id,zbMATH id,first name,middle name,surname,ENTER GENDER HERE
0,1091,zhou.ben-da,ben,da,zhou,u
1,5608,di-castro.agnese,agnese,NaN,di-castro,f
2,15024,szarek.michael,michael,NaN,szarek,m
3,20764,watanabe.yumiko,yumiko,NaN,watanabe,f
4,34006,liu.shaoqing,shaoqing,NaN,liu,m


In [5]:
test_data_algo.head()

,id,zbMATH id,first name,middle name,surname,GENDER FROM ALGORITHM
0,1091,zhou.ben-da,ben,da,zhou,u
1,5608,di-castro.agnese,agnese,NaN,di-castro,f
2,15024,szarek.michael,michael,NaN,szarek,m
3,20764,watanabe.yumiko,yumiko,NaN,watanabe,f
4,34006,liu.shaoqing,shaoqing,NaN,liu,u


In [19]:
test_data_zbmath = test_data_manual.merge(test_data_algo[["id", "GENDER FROM ALGORITHM"]], on="id")

In [20]:
test_data_zbmath.head()

,id,zbMATH id,first name,middle name,surname,ENTER GENDER HERE,GENDER FROM ALGORITHM
0,1091,zhou.ben-da,ben,da,zhou,u,u
1,5608,di-castro.agnese,agnese,NaN,di-castro,f,f
2,15024,szarek.michael,michael,NaN,szarek,m,m
3,20764,watanabe.yumiko,yumiko,NaN,watanabe,f,f
4,34006,liu.shaoqing,shaoqing,NaN,liu,m,u


In [21]:
test_data_zbmath = test_data_zbmath.rename(columns={"GENDER FROM ALGORITHM": "gender_infered", 
                                                    "ENTER GENDER HERE": "gender", 
                                                    "first name": "first_name",
                                                    "middle name": "middle_name",
                                                    "surname": "last_name"})

In [22]:
test_data_zbmath.head()

,id,zbMATH id,first_name,middle_name,last_name,gender,gender_infered
0,1091,zhou.ben-da,ben,da,zhou,u,u
1,5608,di-castro.agnese,agnese,NaN,di-castro,f,f
2,15024,szarek.michael,michael,NaN,szarek,m,m
3,20764,watanabe.yumiko,yumiko,NaN,watanabe,f,f
4,34006,liu.shaoqing,shaoqing,NaN,liu,m,u


### Store in CSV files 

In [26]:
# This file can be used to evaluate other name-based inference services
test_data_zbmath.to_csv("test_data_zbmath.csv", 
                        columns=["first_name", "middle_name", "last_name", "gender"],
                        quoting=csv.QUOTE_ALL,
                       index=False)

In [27]:
# This file can be used to evaluate the name list by Joerg Michael
test_data_zbmath.to_csv("test_data_zbmath_joergmichael.csv", 
                        columns=["first_name", "middle_name", "last_name", "gender", "gender_infered"],
                        quoting=csv.QUOTE_ALL,
                       index=False)